In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when,lit,current_timestamp

In [0]:
%run /Workspace/Users/bishalbishal6659@gmail.com/utilited/utilites

In [0]:
print(bronze_schema)

In [0]:
dbutils.widgets.text("catalog","fmcg","catalog")
dbutils.widgets.text("data_source","customers","data Source")

In [0]:
%python
catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")
base_path = f"s3://sportsbar-dp-vishal/{data_source}/*.csv"
print(base_path)

In [0]:
%python
from pyspark.sql.functions import current_timestamp

df = (
    spark.read.format("csv")
    .option("inferSchema", True)
    .load(base_path)
    .withColumn("Read_timestamp", current_timestamp())
    .withColumnRenamed("_c1", "name")
    .select(
        "*",
        "_metadata.file_name",
        "_metadata.file_size"
    )
)
display(df)

In [0]:
df.write\
    .format("delta")\
    .option("delta.enableChangeDataFeed", "true")\
    .mode("append")\
    .saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")

##data cleaning for Silver leyar

In [0]:
df_bronze =spark.sql(f"SELECT * FROM {catalog}.{bronze_schema}.{data_source};")
display(df_bronze)



In [0]:
#duplicate
df_duplicate_number = df_bronze.groupBy("_c0").count().filter(col("count")>1)
display(df_duplicate_number)
#total number of col before and after 
print(f"the number of count col: ",df_bronze.count())
print(f"rows after cout,{df_duplicate_number.count()}")